<a href="https://colab.research.google.com/github/lilianli17/movie-rec-engine/blob/main/CIS_5500_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from functools import partial
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# TMDB dataset: 4800+ movies
# df_tmbd_movies = pd.read_csv("/content/drive/MyDrive/TMDB 5000 Movie Dataset/tmdb_5000_credits.csv")
# df_tmbd_credits = pd.read_csv("/content/drive/MyDrive/TMDB 5000 Movie Dataset/tmdb_5000_movies.csv")

**movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

**credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

**links.csv**: The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

**links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

**ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

In [3]:
# movies: 43373
df_movies_meta = pd.read_csv("/content/drive/MyDrive/The Movies Dataset/movies_metadata.csv")
df_keywords = pd.read_csv("/content/drive/MyDrive/The Movies Dataset/keywords.csv")
df_credits = pd.read_csv("/content/drive/MyDrive/The Movies Dataset/credits.csv")
df_links = pd.read_csv("/content/drive/MyDrive/The Movies Dataset/links.csv")

<ipython-input-3-8596565d0450>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_meta = pd.read_csv("/content/drive/MyDrive/The Movies Dataset/movies_metadata.csv")


In [33]:
df_links['tmdbId'] = df_links['tmdbId'].fillna(0).astype(int)

In [44]:
df_movies_cleaned = df_movies_meta[(df_movies_meta['status'] == 'Released')][[
    'genres', 'belongs_to_collection', 'id', 'imdb_id', 'original_language', 
    'original_title', 'overview', 'popularity', 'release_date', 'runtime', ]]
df_movies_cleaned = df_movies_cleaned.dropna(subset=['release_date', 'original_title', 'imdb_id', 
                                 'runtime', 'overview', 'original_language', 'popularity'])

In [6]:
def func(col, str):
  exec("global arr; arr = " + str)
  return [x[col] for x in arr] # type: ignore

In [7]:
df_generes = df_movies_cleaned[['genres', 'id', 'original_title']]
exec("global columns; columns = ['name']")
for col in columns: # type: ignore
  df_generes[col] = df_generes["genres"].apply(partial(func, col))
df_generes = df_generes.drop("genres", axis = 1).explode("name").rename({"name": "genre"}, axis = 1).reset_index(drop = True)
df_generes

<ipython-input-7-887111111961>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_generes[col] = df_generes["genres"].apply(partial(func, col))


,id,original_title,genre
0,862,Toy Story,Animation
1,862,Toy Story,Comedy
2,862,Toy Story,Family
3,8844,Jumanji,Adventure
4,8844,Jumanji,Fantasy
...,...,...,...
91305,67758,Betrayal,Action
91306,67758,Betrayal,Drama
91307,67758,Betrayal,Thriller
91308,227506,Satana likuyushchiy,NaN


In [86]:
df_generes.dropna(subset = ['genre'], inplace = True)
df_generes['id'] = df_generes['id'].astype(int)
df_generes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89219 entries, 0 to 91307
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              89219 non-null  int64 
 1   original_title  89219 non-null  object
 2   genre           89219 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [8]:
df_collections = df_movies_cleaned[(df_movies_cleaned['belongs_to_collection'].notnull())][['id', 'original_title', 'belongs_to_collection']]
df_collections['collection'] = df_collections['belongs_to_collection'].apply(lambda x: func("name", "[" + x + "]"))
df_collections = df_collections.drop('belongs_to_collection', axis = 1).explode('collection').reset_index(drop = True)
df_collections

,id,original_title,collection
0,862,Toy Story,Toy Story Collection
1,15602,Grumpier Old Men,Grumpy Old Men Collection
2,11862,Father of the Bride Part II,Father of the Bride Collection
3,710,GoldenEye,James Bond Collection
4,21032,Balto,Balto Collection
...,...,...,...
4401,24568,Follow That Camel,The Carry On Collection
4402,19307,Carry On Camping,The Carry On Collection
4403,21251,Carry On England,The Carry On Collection
4404,460135,LEGO DC Super Hero Girls: Brain Drain,DC Super Hero Girls Collection


In [88]:
df_collections['id'] = df_collections['id'].astype(int)

In [45]:
df_movies_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44010 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genres                 44010 non-null  object 
 1   belongs_to_collection  4406 non-null   object 
 2   id                     44010 non-null  object 
 3   imdb_id                44010 non-null  object 
 4   original_language      44010 non-null  object 
 5   original_title         44010 non-null  object 
 6   overview               44010 non-null  object 
 7   popularity             44010 non-null  object 
 8   release_date           44010 non-null  object 
 9   runtime                44010 non-null  float64
dtypes: float64(1), object(9)
memory usage: 3.7+ MB


In [46]:
df_movies_cleaned = df_movies_cleaned.drop(columns=['genres', 'belongs_to_collection'])
df_movies_cleaned['id'] = df_movies_cleaned['id'].astype(int)
df_movies_cleaned['imdb_id'] = df_movies_cleaned['imdb_id'].apply(lambda x: x[2:]).astype(int)
df_movies_cleaned['original_language'] = df_movies_cleaned['original_language'].astype(str)
df_movies_cleaned['overview'] = df_movies_cleaned['overview'].astype(str)
df_movies_cleaned['popularity'] = df_movies_cleaned['popularity'].astype(float)
df_movies_cleaned['release_date'] = df_movies_cleaned['release_date'].astype('datetime64[ns]')

df_movies_cleaned

,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime
0,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0
1,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,104.0
2,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,101.0
3,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,127.0
4,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,106.0
...,...,...,...,...,...,...,...,...
45460,30840,102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,104.0
45462,111109,2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,360.0
45463,67758,303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,90.0
45464,227506,8536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,87.0


In [9]:
df_keywords["keywords"] = df_keywords["keywords"].apply(partial(func, "name"))
df_keywords = df_keywords.explode("keywords")
df_keywords

,id,keywords
0,862,jealousy
0,862,toy
0,862,boy
0,862,friendship
0,862,friends
...,...,...
46415,111109,play
46415,111109,pinoy
46416,67758,NaN
46417,227506,NaN


In [98]:
df_credits_cast = df_credits[['cast', 'id']]
df_credits_crew = df_credits[['crew', 'id']]

In [99]:
exec("global columns; columns = list(" + df_credits_crew['crew'].iloc[0] + "[0].keys())")
for col in columns: # type: ignore
  df_credits_crew[col] = df_credits_crew["crew"].apply(partial(func, col))
df_credits_crew = df_credits_crew.drop("crew", axis = 1).explode(columns) # type: ignore 
df_credits_crew

,id,credit_id,department,gender,job,name,profile_path
0,7879,52fe4284c3a36847f8024f49,Directing,2,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
0,12891,52fe4284c3a36847f8024f4f,Writing,2,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
0,7,52fe4284c3a36847f8024f55,Writing,2,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
0,12892,52fe4284c3a36847f8024f5b,Writing,2,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
0,12893,52fe4284c3a36847f8024f61,Writing,0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
45473,549356,52fe4776c3a368484e0c8399,Sound,0,Original Music Composer,Richard McHugh,None
45473,58818,52fe4776c3a368484e0c839f,Camera,2,Director of Photography,João Fernandes,None
45474,1085341,533bccebc3a36844cf0011a7,Directing,0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
45474,1195656,58ebbc26925141281908aa0a,Production,2,Producer,Joseph N. Ermolieff,None


In [100]:
df_credits_crew.dropna(subset=['id', 'credit_id', 'department', 'job', 'name'], inplace = True)
df_credits_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464314 entries, 0 to 45475
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            464314 non-null  object
 1   credit_id     464314 non-null  object
 2   department    464314 non-null  object
 3   gender        464314 non-null  object
 4   job           464314 non-null  object
 5   name          464314 non-null  object
 6   profile_path  95098 non-null   object
dtypes: object(7)
memory usage: 28.3+ MB


In [101]:
df_credits_crew = df_credits_crew.drop(['profile_path'], axis=1)

df_credits_crew['gender'] = df_credits_crew['gender'].astype(int)
df_credits_crew['id'] = df_credits_crew['id'].astype(int)

df_credits_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464314 entries, 0 to 45475
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          464314 non-null  int64 
 1   credit_id   464314 non-null  object
 2   department  464314 non-null  object
 3   gender      464314 non-null  int64 
 4   job         464314 non-null  object
 5   name        464314 non-null  object
dtypes: int64(2), object(4)
memory usage: 24.8+ MB


In [104]:
exec("global columns; columns = list(" + df_credits_cast['cast'].iloc[0] + "[0].keys())")
for col in columns: # type: ignore
  df_credits_cast[col] = df_credits_cast["cast"].apply(partial(func, col))
df_credits_cast = df_credits_cast.drop("cast", axis = 1).explode(columns) # type: ignore 
df_credits_cast

,id,cast_id,character,credit_id,gender,name,order,profile_path
0,31,14,Woody (voice),52fe4284c3a36847f8024f95,2,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
0,12898,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
0,7167,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
0,12899,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
0,12900,18,Rex (voice),52fe4284c3a36847f8024fa5,2,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
45474,1090923,3,,52fe4ea59251416c7515d7d9,1,Nathalie Lissenko,1,None
45474,1136422,4,,52fe4ea59251416c7515d7dd,2,Pavel Pavlov,2,None
45474,1261758,5,,52fe4ea59251416c7515d7e1,0,Aleksandr Chabrov,3,None
45474,29199,6,,52fe4ea59251416c7515d7e5,1,Vera Orlova,4,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [111]:
df_credits_cast.dropna(inplace = True)
df_credits_cast['cast_id'] = df_credits_cast['cast_id'].astype(int)
df_credits_cast['id'] = df_credits_cast['id'].astype(int)
df_credits_cast['gender'] = df_credits_cast['gender'].astype(int)
df_credits_cast['order'] = df_credits_cast['order'].astype(int)
df_credits_cast = df_credits_cast.drop('profile_path', axis = 1)
df_credits_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388618 entries, 0 to 45474
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         388618 non-null  int64 
 1   cast_id    388618 non-null  int64 
 2   character  388618 non-null  object
 3   credit_id  388618 non-null  object
 4   gender     388618 non-null  int64 
 5   name       388618 non-null  object
 6   order      388618 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 31.8+ MB


In [135]:
# output csv files
data_dir = '/content/drive/MyDrive/550 datesets'

df_movies_cleaned.to_csv(data_dir + '/df_movies_cleaned.csv')
df_generes.to_csv(data_dir + '/df_generes.csv')
df_collections.to_csv(data_dir + '/df_collections.csv')
df_keywords.to_csv(data_dir + '/df_keywords.csv')
df_credits_cast.to_csv(data_dir + '/df_credits_cast.csv')
df_credits_crew.to_csv(data_dir + '/df_credits_crew.csv')
df_links.to_csv(data_dir + '/df_links.csv')

In [90]:
df_movies_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44010 entries, 0 to 45465
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 44010 non-null  int64         
 1   imdb_id            44010 non-null  int64         
 2   original_language  44010 non-null  object        
 3   original_title     44010 non-null  object        
 4   overview           44010 non-null  object        
 5   popularity         44010 non-null  float64       
 6   release_date       44010 non-null  datetime64[ns]
 7   runtime            44010 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 3.0+ MB


In [115]:
df_movies_cleaned.head(2)

,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime
0,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0
1,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,104.0


In [116]:
df_links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844


In [91]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movieId  45843 non-null  int64
 1   imdbId   45843 non-null  int64
 2   tmdbId   45843 non-null  int64
dtypes: int64(3)
memory usage: 1.0 MB


In [117]:
df_keywords.head(2)

,id,keywords
0,862,jealousy
0,862,toy


In [134]:
df_keywords.dropna(inplace = True)
df_keywords = df_keywords.drop_duplicates()
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156602 entries, 0 to 46415
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        156602 non-null  int64 
 1   keywords  156602 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [133]:
df_generes = df_generes.drop_duplicates()
df_generes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89142 entries, 0 to 91307
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              89142 non-null  int64 
 1   original_title  89142 non-null  object
 2   genre           89142 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [132]:
df_collections = df_collections.drop_duplicates()
df_collections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4403 entries, 0 to 4405
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              4403 non-null   int64 
 1   original_title  4403 non-null   object
 2   collection      4403 non-null   object
dtypes: int64(1), object(2)
memory usage: 137.6+ KB


In [130]:
len(set(df_credits_cast['credit_id']))

388366

In [131]:
df_credits_cast = df_credits_cast.drop_duplicates()
df_credits_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388366 entries, 0 to 45474
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         388366 non-null  int64 
 1   cast_id    388366 non-null  int64 
 2   character  388366 non-null  object
 3   credit_id  388366 non-null  object
 4   gender     388366 non-null  int64 
 5   name       388366 non-null  object
 6   order      388366 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 23.7+ MB


In [126]:
df_credits_crew = df_credits_crew.drop_duplicates()

In [127]:
df_credits_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463836 entries, 0 to 45475
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          463836 non-null  int64 
 1   credit_id   463836 non-null  object
 2   department  463836 non-null  object
 3   gender      463836 non-null  int64 
 4   job         463836 non-null  object
 5   name        463836 non-null  object
dtypes: int64(2), object(4)
memory usage: 24.8+ MB
